In [1]:
import requests
url="https://raw.githubusercontent.com/Pradeep39/cricket_analytics/master/utilities/cricket_data_wrangling.py"
sc.addPyFile(url)
exec(requests.get(url).text)

In [2]:
pool=ThreadPool()
all_matches_df_list=list()
def getMatchResultCallBack(resultDF):
    if not resultDF.empty:
        all_matches_df_list.append(resultDF)
for i in range(1999, 2020):
    pool.apply_async(getMatchResults, args=(str(i),), callback=getMatchResultCallBack)
pool.close()
pool.join()
all_matches_df = pd.DataFrame({})
for df in all_matches_df_list:
    all_matches_df=all_matches_df.append(df)
    
match_schema = StructType([StructField('Team1', StringType()), StructField('Team2',StringType()),StructField('Winner',StringType()),StructField('Margin',StringType()),StructField('Ground',StringType()),StructField('match_date',StringType()),StructField('match_no',StringType()),StructField('match_url',StringType())])

match_results_spark_df=sqlContext.createDataFrame(all_matches_df,schema=match_schema)
match_results_spark_df.limit(5).toPandas()

,Team1,Team2,Winner,Margin,Ground,match_date,match_no,match_url
0,Asia XI,ICC World XI,ICC World XI,112 runs,Melbourne,"Jan 10, 2005",2203,/ci/engine/match/66387.html
1,Australia,West Indies,Australia,116 runs,Melbourne,"Jan 14, 2005",2204,/ci/engine/match/65657.html
2,Australia,Pakistan,Australia,4 wickets,Hobart,"Jan 16, 2005",2205,/ci/engine/match/65658.html
3,Pakistan,West Indies,Pakistan,6 wickets,Brisbane,"Jan 19, 2005",2206,/ci/engine/match/65659.html
4,Bangladesh,Zimbabwe,Zimbabwe,22 runs,Dhaka,"Jan 20, 2005",2207,/ci/engine/match/64918.html


In [3]:
all_matches_df = pd.DataFrame({})
for df in all_matches_df_list:
    all_matches_df=all_matches_df.append(df)
all_matches_df=all_matches_df.reset_index(drop=True)

pool=ThreadPool()
all_players_df_list=list()
for index, row in all_matches_df.iterrows():
    def getMatchDataCallBack(resultDF):
        if not resultDF.empty:
            all_players_df_list.append(resultDF)
            #print(resultDF)
            #print("###################################")
    pool.apply_async(getMatchData, args=(row['match_url'],), callback=getMatchDataCallBack)
pool.close()
pool.join()


all_players_df=pd.DataFrame({})
for df in all_players_df_list:
    all_players_df=all_players_df.append(df)
all_players_df=all_players_df.reset_index(drop=True)

player_schema = StructType([StructField('player_page_href', StringType()), StructField('team',StringType()),StructField('match_no',StringType()),StructField('match_url',StringType()),StructField('player_profile',StringType())])
players_spark_df=sqlContext.createDataFrame(all_players_df,schema=player_schema)
players_spark_df.limit(5).toPandas()

http://stats.espncricinfo.com/ci/engine/match/65659.html
http://stats.espncricinfo.com/ci/engine/match/65661.html
http://stats.espncricinfo.com/ci/engine/match/65658.html
http://stats.espncricinfo.com/ci/engine/match/65660.html
http://stats.espncricinfo.com/ci/engine/match/66387.html
http://stats.espncricinfo.com/ci/engine/match/64919.html
http://stats.espncricinfo.com/ci/engine/match/65657.html
http://stats.espncricinfo.com/ci/engine/match/64918.html
http://stats.espncricinfo.com/ci/engine/match/64920.html
http://stats.espncricinfo.com/ci/engine/match/65663.html
http://stats.espncricinfo.com/ci/engine/match/65662.html
http://stats.espncricinfo.com/ci/engine/match/64921.html
http://stats.espncricinfo.com/ci/engine/match/65664.html
http://stats.espncricinfo.com/ci/engine/match/64922.html
http://stats.espncricinfo.com/ci/engine/match/64923.html
http://stats.espncricinfo.com/ci/engine/match/65665.html
http://stats.espncricinfo.com/ci/engine/match/64924.html
http://stats.espncricinfo.com/c

http://stats.espncricinfo.com/ci/engine/match/64712.html
http://stats.espncricinfo.com/ci/engine/match/65789.html
http://stats.espncricinfo.com/ci/engine/match/64713.html
http://stats.espncricinfo.com/ci/engine/match/64714.html
http://stats.espncricinfo.com/ci/engine/match/65790.html
http://stats.espncricinfo.com/ci/engine/match/65791.html
http://stats.espncricinfo.com/ci/engine/match/65792.html
http://stats.espncricinfo.com/ci/engine/match/65793.html
http://stats.espncricinfo.com/ci/engine/match/65794.html
http://stats.espncricinfo.com/ci/engine/match/65795.html
http://stats.espncricinfo.com/ci/engine/match/64715.html
http://stats.espncricinfo.com/ci/engine/match/64716.html
http://stats.espncricinfo.com/ci/engine/match/64718.html
http://stats.espncricinfo.com/ci/engine/match/64717.html
http://stats.espncricinfo.com/ci/engine/match/64719.html
http://stats.espncricinfo.com/ci/engine/match/64720.html
http://stats.espncricinfo.com/ci/engine/match/64721.html
http://stats.espncricinfo.com/c

http://stats.espncricinfo.com/ci/engine/match/66308.html
http://stats.espncricinfo.com/ci/engine/match/66309.html
http://stats.espncricinfo.com/ci/engine/match/66310.html
http://stats.espncricinfo.com/ci/engine/match/66311.html
http://stats.espncricinfo.com/ci/engine/match/65706.html
http://stats.espncricinfo.com/ci/engine/match/65707.html
http://stats.espncricinfo.com/ci/engine/match/65708.html
http://stats.espncricinfo.com/ci/engine/match/65709.html
http://stats.espncricinfo.com/ci/engine/match/65710.html
http://stats.espncricinfo.com/ci/engine/match/65711.html
http://stats.espncricinfo.com/ci/engine/match/65712.html
http://stats.espncricinfo.com/ci/engine/match/65713.html
http://stats.espncricinfo.com/ci/engine/match/65714.html
http://stats.espncricinfo.com/ci/engine/match/65715.html
http://stats.espncricinfo.com/ci/engine/match/65716.html
http://stats.espncricinfo.com/ci/engine/match/65717.html
http://stats.espncricinfo.com/ci/engine/match/65718.html
http://stats.espncricinfo.com/c

http://stats.espncricinfo.com/ci/engine/match/66341.html
http://stats.espncricinfo.com/ci/engine/match/66342.html
http://stats.espncricinfo.com/ci/engine/match/66343.html
http://stats.espncricinfo.com/ci/engine/match/66344.html
http://stats.espncricinfo.com/ci/engine/match/66345.html
http://stats.espncricinfo.com/ci/engine/match/66346.html
http://stats.espncricinfo.com/ci/engine/match/66347.html
http://stats.espncricinfo.com/ci/engine/match/66348.html
http://stats.espncricinfo.com/ci/engine/match/66349.html
http://stats.espncricinfo.com/ci/engine/match/66180.html
http://stats.espncricinfo.com/ci/engine/match/66181.html
http://stats.espncricinfo.com/ci/engine/match/66182.html
http://stats.espncricinfo.com/ci/engine/match/66183.html
http://stats.espncricinfo.com/ci/engine/match/66184.html
http://stats.espncricinfo.com/ci/engine/match/66185.html
http://stats.espncricinfo.com/ci/engine/match/66186.html
http://stats.espncricinfo.com/ci/engine/match/66187.html
http://stats.espncricinfo.com/c

http://stats.espncricinfo.com/ci/engine/match/66263.html
http://stats.espncricinfo.com/ci/engine/match/66268.html
http://stats.espncricinfo.com/ci/engine/match/66269.html
http://stats.espncricinfo.com/ci/engine/match/66270.html
http://stats.espncricinfo.com/ci/engine/match/66271.html
http://stats.espncricinfo.com/ci/engine/match/66272.html
http://stats.espncricinfo.com/ci/engine/match/66273.html
http://stats.espncricinfo.com/ci/engine/match/64665.html
http://stats.espncricinfo.com/ci/engine/match/64666.html
http://stats.espncricinfo.com/ci/engine/match/64667.html
http://stats.espncricinfo.com/ci/engine/match/66237.html
http://stats.espncricinfo.com/ci/engine/match/66239.html
http://stats.espncricinfo.com/ci/engine/match/66238.html
http://stats.espncricinfo.com/ci/engine/match/66240.html
http://stats.espncricinfo.com/ci/engine/match/64668.html
http://stats.espncricinfo.com/ci/engine/match/64669.html
http://stats.espncricinfo.com/ci/engine/match/64670.html
http://stats.espncricinfo.com/c

...

http://stats.espncricinfo.com/ci/engine/match/743983.html
http://stats.espncricinfo.com/ci/engine/match/919607.html
http://stats.espncricinfo.com/ci/engine/match/743985.html
http://stats.espncricinfo.com/ci/engine/match/919609.html
http://stats.espncricinfo.com/ci/engine/match/919611.html
http://stats.espncricinfo.com/ci/engine/match/919613.html
http://stats.espncricinfo.com/ci/engine/match/903593.html
http://stats.espncricinfo.com/ci/engine/match/919615.html
http://stats.espncricinfo.com/ci/engine/match/903595.html
http://stats.espncricinfo.com/ci/engine/match/919617.html
http://stats.espncricinfo.com/ci/engine/match/924627.html
http://stats.espncricinfo.com/ci/engine/match/924629.html
http://stats.espncricinfo.com/ci/engine/match/903597.html
http://stats.espncricinfo.com/ci/engine/match/924633.html
http://stats.espncricinfo.com/ci/engine/match/924631.html
http://stats.espncricinfo.com/ci/engine/match/903599.html
http://stats.espncricinfo.com/ci/engine/match/924635.html
http://stats.e

http://stats.espncricinfo.com/ci/engine/match/1130738.html
http://stats.espncricinfo.com/ci/engine/match/1131277.html
http://stats.espncricinfo.com/ci/engine/match/1115806.html
http://stats.espncricinfo.com/ci/engine/match/1072311.html
http://stats.espncricinfo.com/ci/engine/match/1130739.html
http://stats.espncricinfo.com/ci/engine/match/1072312.html
http://stats.espncricinfo.com/ci/engine/match/1131278.html
http://stats.espncricinfo.com/ci/engine/match/1130740.html
http://stats.espncricinfo.com/ci/engine/match/1130741.html
http://stats.espncricinfo.com/ci/engine/match/1131279.html
http://stats.espncricinfo.com/ci/engine/match/1130742.html
http://stats.espncricinfo.com/ci/engine/match/1072313.html
http://stats.espncricinfo.com/ci/engine/match/1130743.html
http://stats.espncricinfo.com/ci/engine/match/1072314.html
http://stats.espncricinfo.com/ci/engine/match/1122279.html
http://stats.espncricinfo.com/ci/engine/match/1122280.html
http://stats.espncricinfo.com/ci/engine/match/1122281.ht

http://stats.espncricinfo.com/ci/engine/match/1158067.html
http://stats.espncricinfo.com/ci/engine/match/1158068.html
http://stats.espncricinfo.com/ci/engine/match/1158069.html
http://stats.espncricinfo.com/ci/engine/match/1168115.html
http://stats.espncricinfo.com/ci/engine/match/1168116.html
http://stats.espncricinfo.com/ci/engine/match/1168242.html
http://stats.espncricinfo.com/ci/engine/match/1158070.html
http://stats.espncricinfo.com/ci/engine/match/1144167.html
http://stats.espncricinfo.com/ci/engine/match/1168117.html
http://stats.espncricinfo.com/ci/engine/match/1168243.html
http://stats.espncricinfo.com/ci/engine/match/1144168.html
http://stats.espncricinfo.com/ci/engine/match/1168118.html
http://stats.espncricinfo.com/ci/engine/match/1168244.html
http://stats.espncricinfo.com/ci/engine/match/1168119.html
http://stats.espncricinfo.com/ci/engine/match/1168245.html
http://stats.espncricinfo.com/ci/engine/match/1144169.html
http://stats.espncricinfo.com/ci/engine/match/1168246.ht

http://stats.espncricinfo.com/ci/engine/match/1031429.html
http://stats.espncricinfo.com/ci/engine/match/1022347.html
http://stats.espncricinfo.com/ci/engine/match/1022349.html
http://stats.espncricinfo.com/ci/engine/match/1022351.html
http://stats.espncricinfo.com/ci/engine/match/1022353.html
http://stats.espncricinfo.com/ci/engine/match/1022355.html
http://stats.espncricinfo.com/ci/engine/match/1022357.html
http://stats.espncricinfo.com/ci/engine/match/1022359.html
http://stats.espncricinfo.com/ci/engine/match/1022361.html
http://stats.espncricinfo.com/ci/engine/match/1022363.html
http://stats.espncricinfo.com/ci/engine/match/1089779.html
http://stats.espncricinfo.com/ci/engine/match/1022365.html
http://stats.espncricinfo.com/ci/engine/match/1022367.html
http://stats.espncricinfo.com/ci/engine/match/1089780.html
http://stats.espncricinfo.com/ci/engine/match/1022369.html
http://stats.espncricinfo.com/ci/engine/match/1022371.html
http://stats.espncricinfo.com/ci/engine/match/1089781.ht

,player_page_href,team,match_no,match_url,player_profile
0,/ci/content/player/51880.html,West Indies,2206,/ci/engine/match/65659.html,51880
1,/ci/content/player/52057.html,West Indies,2206,/ci/engine/match/65659.html,52057
2,/ci/content/player/52969.html,West Indies,2206,/ci/engine/match/65659.html,52969
3,/ci/content/player/52337.html,West Indies,2206,/ci/engine/match/65659.html,52337
4,/ci/content/player/52681.html,West Indies,2206,/ci/engine/match/65659.html,52681


In [4]:
match_players_spark_df=match_results_spark_df.join(players_spark_df,"match_no")
match_players_spark_df.limit(5).toPandas()

,match_no,Team1,Team2,Winner,Margin,Ground,match_date,match_url,player_page_href,team,match_url,player_profile
0,1436,West Indies,Australia,Australia,46 runs,St George's,"Apr 14, 1999",/ci/engine/match/64619.html,/ci/content/player/5390.html,Australia,/ci/engine/match/64619.html,5390
1,1436,West Indies,Australia,Australia,46 runs,St George's,"Apr 14, 1999",/ci/engine/match/64619.html,/ci/content/player/8189.html,Australia,/ci/engine/match/64619.html,8189
2,1436,West Indies,Australia,Australia,46 runs,St George's,"Apr 14, 1999",/ci/engine/match/64619.html,/ci/content/player/6513.html,Australia,/ci/engine/match/64619.html,6513
3,1436,West Indies,Australia,Australia,46 runs,St George's,"Apr 14, 1999",/ci/engine/match/64619.html,/ci/content/player/6285.html,Australia,/ci/engine/match/64619.html,6285
4,1436,West Indies,Australia,Australia,46 runs,St George's,"Apr 14, 1999",/ci/engine/match/64619.html,/ci/content/player/8192.html,Australia,/ci/engine/match/64619.html,8192


In [5]:
player_profile_set = set(all_players_df['player_profile'].tolist())
    
pool=ThreadPool()
players_career_summ_df_list=list()

for player_profile in player_profile_set:
    def getPlayerSummaryCallBack(resultDF):
        if not resultDF.empty:
            players_career_summ_df_list.append(resultDF)
            #print(resultDF)
            #print("###################################")
    pool.apply_async(getPlayerSummary, args=(player_profile,), callback=getPlayerSummaryCallBack)
pool.close()
pool.join()

players_career_summ_df = pd.DataFrame({})
for df in players_career_summ_df_list:
    players_career_summ_df=players_career_summ_df.append(df)

players_career_summ_df=players_career_summ_df.reset_index(drop=True)
players_career_summ_df[:5]

,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,...,b_Wkts,b_BBI,b_BBM,b_Ave,b_Econ,b_SR,b_4w,b_5w,b_10,player_profile
0,30,22,6,161,41,10.06,218,73.85,0,0,...,33,4/23,4/23,30.54,4.67,39.2,3,0,0,24943
1,8,8,6,30,7*,15,61,49.18,0,0,...,5,1/35,1/35,67.40,6.8,59.4,0,0,0,465783
2,25,6,4,27,13*,13.5,46,58.69,0,0,...,35,5/22,5/22,23.25,4.05,34.3,1,2,0,8231
3,1,1,1,0,0*,-,0,-,0,0,...,1,1/40,1/40,40,10,24,0,0,0,355267
4,27,18,4,110,18,7.85,201,54.72,0,0,...,51,5/46,5/46,26.66,5.87,27.2,2,1,0,348059


In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

player_schema = StructType([StructField("Mat", StringType(), True),StructField("Inns", StringType(), True),StructField("NO", StringType(), True),StructField("Runs", StringType(), True),StructField("HS", StringType(), True),StructField("Ave", StringType(), True),StructField("BF", StringType(), True),StructField("SR", StringType(), True),StructField("bat_100", StringType(), True),StructField("bat_50", StringType(), True),StructField("bat_4s", StringType(), True),StructField("bat_6s", StringType(), True),StructField("Ct", StringType(), True),StructField("St", StringType(), True),StructField("b_Mat", StringType(), True),StructField("b_Inns", StringType(), True),StructField("b_Overs", StringType(), True),StructField("b_Mdns", StringType(), True),StructField("b_Runs", StringType(), True),StructField("b_Wkts", StringType(), True),StructField("b_BBI", StringType(), True),StructField("b_BBM", StringType(), True),StructField("b_Ave", StringType(), True),StructField("b_Econ", StringType(), True),StructField("b_SR", StringType(), True),StructField("b_4w", StringType(), True),StructField("b_5w", StringType(), True),StructField("player_profile", StringType(), True)])

players_spark_df=sqlContext.createDataFrame(players_career_summ_df,schema=player_schema)

clean_players_spark_df = players_spark_df.select(*(regexp_replace(col(c),'-','0').alias(c) for c in players_spark_df.columns))

#clean_bowling_spark_df=clean_bowling_spark_df.select(*(col(c).alias('b_'+c) for c in clean_bowling_spark_df.columns))

all_player_match_spark_df=match_players_spark_df.join(clean_players_spark_df,"player_profile")

all_player_match_spark_df.createOrReplaceTempView("all_player_match_data")
all_player_match_spark_df.limit(5).toPandas()

,player_profile,match_no,Team1,Team2,Winner,Margin,Ground,match_date,match_url,player_page_href,...,b_Mdns,b_Runs,b_Wkts,b_BBI,b_BBM,b_Ave,b_Econ,b_SR,b_4w,b_5w
0,15555,2258,England,Australia,tied,NaN,Lord's,"Jul 2, 2005",/ci/engine/match/212452.html,/ci/content/player/15555.html,...,275,7,2/43,2/43,39.28,4.74,49.7,0.0,0,0
1,15555,2256,England,Australia,no result,NaN,Birmingham,"Jun 28, 2005",/ci/engine/match/212020.html,/ci/content/player/15555.html,...,275,7,2/43,2/43,39.28,4.74,49.7,0.0,0,0
2,15555,2259,England,Australia,England,9 wickets,Leeds,"Jul 7, 2005",/ci/engine/match/212795.html,/ci/content/player/15555.html,...,275,7,2/43,2/43,39.28,4.74,49.7,0.0,0,0
3,15555,2260,England,Australia,Australia,7 wickets,Lord's,"Jul 10, 2005",/ci/engine/match/213080.html,/ci/content/player/15555.html,...,275,7,2/43,2/43,39.28,4.74,49.7,0.0,0,0
4,15555,2197,Zimbabwe,England,England,74 runs,Bulawayo,"Dec 5, 2004",/ci/engine/match/64913.html,/ci/content/player/15555.html,...,275,7,2/43,2/43,39.28,4.74,49.7,0.0,0,0


In [7]:
all_df=sqlContext.sql("""select match_no,team,Team1,Team2,Winner,Margin,avg(Mat) as Mat_avg, avg(Inns) as Inns_avg,avg(NO) NO_avg,avg(Runs) Runs_ave,max(CAST(Runs as Double)) Runs_max,avg(HS) HS_avg,max(CAST(HS as Double)) HS_max, avg(Ave) Ave_avg, max(CAST(Ave as Double)) Ave_max, avg(BF) BF_avg,avg(SR) SR_avg,max(CAST(SR as Double)) SR_max,avg(bat_100) bat_100_avg,max(bat_100) bat_100_max,avg(bat_50) bat_50_ave,max(bat_50) bat_50_max,avg(bat_4s) bat_4s_avg,max(bat_4s) bat_4s_max, avg(bat_6s) bat_6s_avg,max(bat_6s) bat_6s_max,avg(Ct)  Ct_ave,max(CAST(Ct as Double))  Ct_max,avg(St)  St_Avg,max(CAST(St as Double))  St_max,avg(b_Mat) bowl_Mat_Avg,avg(b_Inns) as  bowl_Inns_avg,avg(b_Mdns) bowl_maidens_avg,max(CAST(b_Mdns as Double)) bowl_maidens_max,avg(b_Runs) bowl_Runs_avg,max(CAST(b_Runs as Double)) bowl_Runs_max,avg(b_Wkts) bowl_Wkts_avg,max(CAST(b_Wkts as Double)) bowl_Wkts_max,avg(b_Ave) b_Ave_Avg,avg(b_Econ) bowl_Econ_Avg,min(CAST(b_Econ as Double)) bowl_Econ_min,avg(b_SR) bowl_SR_Avg,min(cast(b_SR as Double)) bowl_SR_min,avg(b_4w) as bowl_4w_avg,max(CAST(b_4w as Double)) as bowl_4w_max,avg(b_5w) bowl_5w_avg, max(CAST(b_5w as Double)) bowl_5w_max, IF(team==winner, 1, 0) label
 from all_player_match_data group by match_no,team,Team1,Team2,Winner,Margin order by match_no,team""")
 
all_df=all_df.cache()
all_df.createOrReplaceTempView("all_aggregated_data")
all_df.limit(5).toPandas()

,match_no,team,Team1,Team2,Winner,Margin,Mat_avg,Inns_avg,NO_avg,Runs_ave,...,b_Ave_Avg,bowl_Econ_Avg,bowl_Econ_min,bowl_SR_Avg,bowl_SR_min,bowl_4w_avg,bowl_4w_max,bowl_5w_avg,bowl_5w_max,label
0,1378,India,New Zealand,India,New Zealand,5 wickets,217.000000,182.090909,30.272727,5525.090909,...,4.451818,44.081818,0.0,1.454545,0.0,1.000000,3.0,0.0,0.0,0
1,1378,New Zealand,New Zealand,India,New Zealand,5 wickets,163.363636,135.000000,25.181818,2978.363636,...,3.914545,39.309091,0.0,1.454545,0.0,0.363636,2.0,0.0,0.0,1
2,1379,Australia,Australia,England,England,7 runs,178.636364,138.454545,25.818182,4201.727273,...,4.172727,43.090909,0.0,2.363636,0.0,0.909091,7.0,0.0,0.0,0
3,1379,England,Australia,England,England,7 runs,76.545455,62.636364,11.636364,1536.909091,...,4.120909,29.518182,0.0,1.363636,0.0,0.454545,2.0,0.0,0.0,1
4,1380,England,England,Sri Lanka,England,4 wickets,76.909091,62.545455,12.000000,1526.090909,...,4.130000,31.927273,0.0,1.363636,0.0,0.454545,2.0,0.0,0.0,1


In [8]:
india_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='India' limit 1""")
pakistan_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk
                        from all_aggregated_data where (team=team1 or team=team2) and team='Pakistan' limit 1""")
australia_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='Australia' limit 1""")
newzealand_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='New Zealand' limit 1""")
srilanka_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='Sri Lanka' limit 1""")
west_indies_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='West Indies' limit 1""")
england_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='England' limit 1""")
bangladesh_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='Bangladesh' limit 1""")
southafrica_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='South Africa' limit 1""")
afghanistan_team_df=sqlContext.sql("""select *, rank() over (order by int(match_no) desc) as rnk 
                        from all_aggregated_data where (team=team1 or team=team2) and team='Afghanistan' limit 1""")
all_teams_metrics_df=india_team_df.union(pakistan_team_df).union(afghanistan_team_df)\
                        .union(australia_team_df).union(newzealand_team_df)\
                        .union(srilanka_team_df).union(west_indies_team_df)\
                        .union(england_team_df).union(bangladesh_team_df)\
                        .union(southafrica_team_df)
all_teams_metrics_df=all_teams_metrics_df.drop('match_no').drop('Winner')\
                        .drop('Margin').drop('label').drop('Team1').drop('Team2')
all_teams_metrics_df.limit(5).toPandas()


,team,Mat_avg,Inns_avg,NO_avg,Runs_ave,Runs_max,HS_avg,HS_max,Ave_avg,Ave_max,...,b_Ave_Avg,bowl_Econ_Avg,bowl_Econ_min,bowl_SR_Avg,bowl_SR_min,bowl_4w_avg,bowl_4w_max,bowl_5w_avg,bowl_5w_max,rnk
0,India,104.727273,81.727273,19.363636,3041.909091,11225.0,104.285714,264.0,29.523636,59.70,...,4.292727,42.918182,0.0,1.636364,0.0,0.454545,2.0,0.0,0.0,1
1,Pakistan,70.000000,58.272727,10.363636,1745.090909,6587.0,110.000000,151.0,35.610000,53.06,...,4.378182,39.809091,0.0,1.181818,0.0,0.272727,1.0,0.0,0.0,1
2,Afghanistan,63.000000,54.363636,7.545455,1232.909091,2697.0,69.000000,116.0,22.398182,34.80,...,4.649091,27.109091,0.0,1.272727,0.0,0.727273,4.0,0.0,0.0,1
3,Australia,66.818182,55.909091,7.818182,1801.000000,4859.0,98.000000,179.0,29.816364,45.41,...,4.587273,31.318182,0.0,1.727273,0.0,0.818182,7.0,0.0,0.0,1
4,New Zealand,89.909091,75.818182,14.000000,2480.000000,8259.0,79.000000,148.0,28.962727,48.06,...,4.343636,26.545455,0.0,1.181818,0.0,0.727273,5.0,0.0,0.0,1


In [9]:
team1_agg_df=sqlContext.sql("select * from all_aggregated_data where team=Team1")
team2_agg_df=sqlContext.sql("select * from all_aggregated_data where team=Team2")
team1_agg_df.createOrReplaceTempView("team1_agg_df")
team2_agg_df.createOrReplaceTempView("team2_agg_df")
match_team_diff_df=sqlContext.sql("select t1.match_no,t1.team,t1.Team1,t1.Team2,t1.Winner,t1.Margin,t1.Mat_avg-t2.Mat_avg diff_Mat_avg,t1.Inns_avg - t2.Inns_avg diff_Inns_avg,t1.NO_avg - t2.NO_avg as diff_NO_avg,t1.Runs_ave-t2.Runs_ave diff_Runs_avg, t1.Runs_max-t2.Runs_max	diff_Runs_max,t1.HS_avg-t2.HS_avg diff_HS_avg,t1.HS_max-t2.HS_max diff_HS_max,t1.Ave_max	-	t2.	Ave_max	diff_Ave_max,t1.Ave_avg	-	t2.	Ave_avg	diff_Ave_avg,t1.BF_avg	-	t2.	BF_avg	diff_BF_avg,t1.SR_avg	-	t2.	SR_avg	diff_SR_avg,t1.SR_max	-	t2.	SR_max	diff_SR_max,t1.bat_100_avg	-	t2.bat_100_avg	diff_100_avg,t1.bat_100_max	-	t2.bat_100_max	diff_100_max,t1.bat_50_ave	-	t2.bat_50_ave	diff_50_ave,t1.bat_50_max	-	t2.bat_50_max	diff_50_max,t1.bat_4s_avg	-	t2.bat_4s_avg	diff_4s_avg,t1.bat_4s_max	-	t2.bat_4s_max	diff_4s_max,t1.bat_6s_avg	-	t2.bat_6s_avg	diff_6s_avg,t1.bat_6s_max	-	t2.bat_6s_max	diff_6s_max,t1.Ct_ave	-	t2.	Ct_ave	diff_Ct_ave,t1.Ct_max	-	t2.	Ct_max	diff_Ct_max,t1.St_Avg	-	t2.	St_Avg	diff_St_Avg,t1.St_max	-	t2.	St_max	diff_St_max,t1.bowl_Mat_Avg	-	t2.	bowl_Mat_Avg	diff_bowl_Mat_Avg,t1.bowl_Inns_avg	-	t2.	bowl_Inns_avg	diff_bowl_Inns_avg,t1.bowl_maidens_avg	-	t2.	bowl_maidens_avg	diff_bowl_maidens_avg,t1.bowl_maidens_max	-	t2.	bowl_maidens_max	diff_bowl_maidens_max,t1.bowl_Runs_avg	-	t2.	bowl_Runs_avg	diff_bowl_Runs_avg,t1.bowl_Runs_max	-	t2.	bowl_Runs_max	diff_bowl_Runs_max,t1.bowl_Wkts_avg	-	t2.	bowl_Wkts_avg	diff_bowl_Wkts_avg,t1.bowl_Wkts_max	-	t2.	bowl_Wkts_max	diff_bowl_Wkts_max,t1.b_Ave_Avg	-	t2.	b_Ave_Avg	diff_b_Ave_Avg,t1.bowl_Econ_Avg	-	t2.	bowl_Econ_Avg	diff_bowl_Econ_Avg,t1.bowl_Econ_min	-	t2.	bowl_Econ_min	diff_bowl_Econ_min,t1.bowl_SR_Avg	-	t2.	bowl_SR_Avg	diff_bowl_SR_Avg,t1.bowl_SR_min	-	t2.	bowl_SR_min	diff_bowl_SR_min,t1.bowl_4w_avg	-	t2.	bowl_4w_avg	diff_bowl_4w_avg,t1.bowl_4w_max	-	t2.	bowl_4w_max	diff_bowl_4w_max,t1.bowl_5w_avg	-	t2.	bowl_5w_avg	diff_bowl_5w_avg,t1.bowl_5w_max	-	t2.	bowl_5w_max	diff_bowl_5w_max,t1.label from team1_agg_df t1,team2_agg_df t2 where t1.match_no=t2.match_no")
match_team_diff_df.limit(5).toPandas()

,match_no,team,Team1,Team2,Winner,Margin,diff_Mat_avg,diff_Inns_avg,diff_NO_avg,diff_Runs_avg,...,diff_b_Ave_Avg,diff_bowl_Econ_Avg,diff_bowl_Econ_min,diff_bowl_SR_Avg,diff_bowl_SR_min,diff_bowl_4w_avg,diff_bowl_4w_max,diff_bowl_5w_avg,diff_bowl_5w_max,label
0,1378,New Zealand,New Zealand,India,New Zealand,5 wickets,-53.636364,-47.090909,-5.090909,-2546.727273,...,-0.537273,-4.772727,0.0,0.000000,0.0,-0.636364,-1.0,0.0,0.0,1
1,1379,Australia,Australia,England,England,7 runs,102.090909,75.818182,14.181818,2664.818182,...,0.051818,13.572727,0.0,1.000000,0.0,0.454545,5.0,0.0,0.0,0
2,1380,England,England,Sri Lanka,England,4 wickets,-162.363636,-128.181818,-21.727273,-2754.818182,...,-1.500909,0.781818,0.0,-2.272727,0.0,-1.363636,-8.0,0.0,0.0,1
3,1381,New Zealand,New Zealand,India,India,2 wickets,-94.909091,-76.272727,-10.727273,-3194.272727,...,-1.045455,-7.681818,0.0,-0.454545,0.0,-0.818182,-1.0,0.0,0.0,0
4,1382,Australia,Australia,Sri Lanka,Australia,8 wickets,-94.909091,-86.818182,-10.181818,-1202.272727,...,-1.314545,25.536364,0.0,-1.090909,0.0,-0.818182,-3.0,0.0,0.0,1


In [127]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import PCA

featureAttributeArray=['diff_Mat_avg', 'diff_Inns_avg', 'diff_NO_avg', 'diff_Runs_avg', 'diff_Runs_max', 'diff_HS_avg', 'diff_HS_max', 'diff_Ave_avg', 'diff_Ave_max', 'diff_BF_avg', 'diff_SR_avg', 'diff_SR_max', 'diff_100_avg', 'diff_100_max', 'diff_50_ave', 'diff_50_max', 'diff_4s_avg', 'diff_4s_max', 'diff_6s_avg', 'diff_6s_max', 'diff_Ct_ave', 'diff_Ct_max', 'diff_St_Avg', 'diff_St_max', 'diff_bowl_Mat_Avg', 'diff_bowl_Inns_avg', 'diff_bowl_maidens_avg', 'diff_bowl_maidens_max', 'diff_bowl_Runs_avg', 'diff_bowl_Runs_max', 'diff_bowl_Wkts_avg', 'diff_bowl_Wkts_max', 'diff_b_Ave_Avg', 'diff_bowl_Econ_Avg', 'diff_bowl_Econ_min', 'diff_bowl_SR_Avg', 'diff_bowl_SR_min', 'diff_bowl_4w_avg', 'diff_bowl_4w_max', 'diff_bowl_5w_avg', 'diff_bowl_5w_max']

assembler = VectorAssembler(
    inputCols=featureAttributeArray,
    outputCol="features", handleInvalid="skip")
                         
pca = PCA(k=25, inputCol="features", outputCol="pcaFeatures")
    
#lr = LogisticRegression(labelCol="label", featuresCol="pcaFeatures", maxIter=50, regParam=0.01)
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=50, regParam=0.01)
                                   
(trainingData, testData) = match_team_diff_df.randomSplit([0.8, 0.2], seed=11)

lr_Pipeline=Pipeline().setStages([assembler, lr])

lr_PipelineModel = lr_Pipeline.fit(trainingData)

predictions = lr_PipelineModel.transform(testData)

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print(evaluator.evaluate(predictions))

evaluator.setMetricName("areaUnderPR")

print(evaluator.evaluate(predictions))

0.747513776752531
0.7763456882455888


In [29]:
import datetime

now=datetime.datetime.now().date()
df_list=pd.read_html("https://www.icccricketschedule.com/icc-world-cup-2019-schedule/")
df=df_list[0]
df['Date']=df['Date'].str.replace('- ','-')
df['Date']=pd.to_datetime(df['Date'], format='%B %d-%Y')
filtered_df=df[(~df['Match Center'].str.contains('Reserve'))]
filtered_df=filtered_df[df['Date']>now]
knockoffs_df=filtered_df[(df['No']>=46) & (df['No']<49)]
knockoffs_df=knockoffs_df.reset_index(drop=True)
final_df=filtered_df[df['No']==50]
final_df=final_df.reset_index(drop=True)
filtered_df=filtered_df[df['No']<46]
teams=filtered_df['Match Center'].str.split(pat=' vs ',expand=True)
filtered_df['Team1']=teams[0]
filtered_df['Team2']=teams[1]

to_be_played_match_df=sqlContext.createDataFrame(filtered_df)
to_be_played_team1_df=to_be_played_match_df.join(all_teams_metrics_df,to_be_played_match_df.Team1 == all_teams_metrics_df.team,how='inner')
to_be_played_team2_df=to_be_played_match_df.join(all_teams_metrics_df,to_be_played_match_df.Team2 == all_teams_metrics_df.team,how='inner')
to_be_played_team1_df.createOrReplaceTempView("to_be_played_team1_data")
to_be_played_team2_df.createOrReplaceTempView("to_be_played_team2_data")
to_be_played_team_diff_df=sqlContext.sql("select t1.Date, t1.Venue, t1.No as match_no,t1.team,t1.Team1,t1.Team2,t1.Mat_avg-t2.Mat_avg diff_Mat_avg,t1.Inns_avg - t2.Inns_avg diff_Inns_avg,t1.NO_avg - t2.NO_avg as diff_NO_avg,t1.Runs_ave-t2.Runs_ave diff_Runs_avg, t1.Runs_max-t2.Runs_max	diff_Runs_max,t1.HS_avg-t2.HS_avg diff_HS_avg,t1.HS_max-t2.HS_max diff_HS_max,t1.Ave_max	-	t2.	Ave_max	diff_Ave_max,t1.Ave_avg	-	t2.	Ave_avg	diff_Ave_avg,t1.BF_avg	-	t2.	BF_avg	diff_BF_avg,t1.SR_avg	-	t2.	SR_avg	diff_SR_avg,t1.SR_max	-	t2.	SR_max	diff_SR_max,t1.bat_100_avg	-	t2.bat_100_avg	diff_100_avg,t1.bat_100_max	-	t2.bat_100_max	diff_100_max,t1.bat_50_ave	-	t2.bat_50_ave	diff_50_ave,t1.bat_50_max	-	t2.	bat_50_max	diff_50_max,t1.bat_4s_avg	-	t2.bat_4s_avg	diff_4s_avg,t1.bat_4s_max	-	t2.bat_4s_max	diff_4s_max,t1.bat_6s_avg	-	t2.bat_6s_avg	diff_6s_avg,t1.bat_6s_max	-	t2.bat_6s_max	diff_6s_max,t1.Ct_ave	-	t2.	Ct_ave	diff_Ct_ave,t1.Ct_max	-	t2.	Ct_max	diff_Ct_max,t1.St_Avg	-	t2.	St_Avg	diff_St_Avg,t1.St_max	-	t2.	St_max	diff_St_max,t1.bowl_Mat_Avg	-	t2.	bowl_Mat_Avg	diff_bowl_Mat_Avg,t1.bowl_Inns_avg	-	t2.	bowl_Inns_avg	diff_bowl_Inns_avg,t1.bowl_maidens_avg	-	t2.	bowl_maidens_avg	diff_bowl_maidens_avg,t1.bowl_maidens_max	-	t2.	bowl_maidens_max	diff_bowl_maidens_max,t1.bowl_Runs_avg	-	t2.	bowl_Runs_avg	diff_bowl_Runs_avg,t1.bowl_Runs_max	-	t2.	bowl_Runs_max	diff_bowl_Runs_max,t1.bowl_Wkts_avg	-	t2.	bowl_Wkts_avg	diff_bowl_Wkts_avg,t1.bowl_Wkts_max	-	t2.	bowl_Wkts_max	diff_bowl_Wkts_max,t1.b_Ave_Avg	-	t2.	b_Ave_Avg	diff_b_Ave_Avg,t1.bowl_Econ_Avg	-	t2.	bowl_Econ_Avg	diff_bowl_Econ_Avg,t1.bowl_Econ_min	-	t2.	bowl_Econ_min	diff_bowl_Econ_min,t1.bowl_SR_Avg	-	t2.	bowl_SR_Avg	diff_bowl_SR_Avg,t1.bowl_SR_min	-	t2.	bowl_SR_min	diff_bowl_SR_min,t1.bowl_4w_avg	-	t2.	bowl_4w_avg	diff_bowl_4w_avg,t1.bowl_4w_max	-	t2.	bowl_4w_max	diff_bowl_4w_max,t1.bowl_5w_avg	-	t2.	bowl_5w_avg	diff_bowl_5w_avg,t1.bowl_5w_max	-	t2.	bowl_5w_max	diff_bowl_5w_max from to_be_played_team1_data t1,to_be_played_team2_data t2 where t1.No=t2.No order by t1.No")

to_be_played_team_diff_df.toPandas()

/Users/preddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  if __name__ == '__main__':
/Users/preddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/Users/preddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/Users/preddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
/Users/preddy/anaconda3/lib/python3.

,Date,Venue,match_no,team,Team1,Team2,diff_Mat_avg,diff_Inns_avg,diff_NO_avg,diff_Runs_avg,...,diff_bowl_Wkts_max,diff_b_Ave_Avg,diff_bowl_Econ_Avg,diff_bowl_Econ_min,diff_bowl_SR_Avg,diff_bowl_SR_min,diff_bowl_4w_avg,diff_bowl_4w_max,diff_bowl_5w_avg,diff_bowl_5w_max
0,2019-07-01,The Riverside- Chester-le-Street,39,Sri Lanka,Sri Lanka,West Indies,114.363636,85.909091,13.090909,1503.090909,...,0.0,1.560909,-2.527273,0.0,2.363636,0.0,1.454545,6.0,0.0,0.0
1,2019-07-02,Edgbaston- Birmingham,40,Bangladesh,Bangladesh,India,-164.909091,-135.545455,-23.363636,-4828.000000,...,0.0,-0.218182,-9.209091,0.0,-1.000000,0.0,-0.909091,-2.0,0.0,0.0
2,2019-07-03,The Riverside- Chester-le-Street,41,England,England,New Zealand,-86.818182,-72.363636,-13.545455,-1441.454545,...,0.0,0.206364,-9.790909,0.0,-0.090909,0.0,0.090909,0.0,0.0,0.0
3,2019-07-04,Headingley- Leeds,42,Afghanistan,Afghanistan,West Indies,-80.545455,-65.454545,-16.090909,-1888.545455,...,0.0,-0.055455,5.781818,0.0,-0.727273,0.0,-0.272727,-3.0,0.0,0.0
4,2019-07-05,Lord’s- London,43,Pakistan,Pakistan,Bangladesh,165.727273,134.727273,24.545455,3419.454545,...,0.0,0.281818,5.954545,0.0,3.272727,0.0,2.454545,8.0,0.0,0.0
5,2019-07-06,Headingley- Leeds,44,Sri Lanka,Sri Lanka,India,22.272727,8.636364,3.454545,-1244.181818,...,0.0,1.179091,-12.936364,0.0,2.181818,0.0,0.818182,7.0,0.0,0.0
6,2019-07-06,Old Trafford- Manchester,45,Australia,Australia,South Africa,-23.181818,-34.272727,-9.272727,-744.272727,...,0.0,0.833636,16.445455,0.0,0.818182,0.0,-0.363636,1.0,0.0,0.0


In [30]:
to_be_played_team_diff_df.toPandas()

,Date,Venue,match_no,team,Team1,Team2,diff_Mat_avg,diff_Inns_avg,diff_NO_avg,diff_Runs_avg,...,diff_bowl_Wkts_max,diff_b_Ave_Avg,diff_bowl_Econ_Avg,diff_bowl_Econ_min,diff_bowl_SR_Avg,diff_bowl_SR_min,diff_bowl_4w_avg,diff_bowl_4w_max,diff_bowl_5w_avg,diff_bowl_5w_max
0,2019-07-01,The Riverside- Chester-le-Street,39,Sri Lanka,Sri Lanka,West Indies,114.363636,85.909091,13.090909,1503.090909,...,0.0,1.560909,-2.527273,0.0,2.363636,0.0,1.454545,6.0,0.0,0.0
1,2019-07-02,Edgbaston- Birmingham,40,Bangladesh,Bangladesh,India,-164.909091,-135.545455,-23.363636,-4828.000000,...,0.0,-0.218182,-9.209091,0.0,-1.000000,0.0,-0.909091,-2.0,0.0,0.0
2,2019-07-03,The Riverside- Chester-le-Street,41,England,England,New Zealand,-86.818182,-72.363636,-13.545455,-1441.454545,...,0.0,0.206364,-9.790909,0.0,-0.090909,0.0,0.090909,0.0,0.0,0.0
3,2019-07-04,Headingley- Leeds,42,Afghanistan,Afghanistan,West Indies,-80.545455,-65.454545,-16.090909,-1888.545455,...,0.0,-0.055455,5.781818,0.0,-0.727273,0.0,-0.272727,-3.0,0.0,0.0
4,2019-07-05,Lord’s- London,43,Pakistan,Pakistan,Bangladesh,165.727273,134.727273,24.545455,3419.454545,...,0.0,0.281818,5.954545,0.0,3.272727,0.0,2.454545,8.0,0.0,0.0
5,2019-07-06,Headingley- Leeds,44,Sri Lanka,Sri Lanka,India,22.272727,8.636364,3.454545,-1244.181818,...,0.0,1.179091,-12.936364,0.0,2.181818,0.0,0.818182,7.0,0.0,0.0
6,2019-07-06,Old Trafford- Manchester,45,Australia,Australia,South Africa,-23.181818,-34.272727,-9.272727,-744.272727,...,0.0,0.833636,16.445455,0.0,0.818182,0.0,-0.363636,1.0,0.0,0.0


In [128]:
predictions = lr_PipelineModel.transform(to_be_played_team_diff_df)
predictions_df=predictions.select("Date","Venue","match_no","Team1","Team2","rawPrediction","probability", "prediction")
win_predictions=predictions_df.toPandas()
win_list1=win_predictions[(win_predictions.prediction==0.0)]['Team2'].value_counts()
win_list2=win_predictions[(win_predictions.prediction==1.0)]['Team1'].value_counts()
win_list=win_list1.append(win_list2)

In [129]:
df_list = pd.read_html("https://www.cricketworldcup.com/")
points_table_df=df_list[0]
points_table_df['Team'], points_table_df['Team_Cd'] = points_table_df['Team'].str.rsplit(n=1).str
points_table_df['Projected PTS']=points_table_df['PTS']
points_table_df

,Pos,Team,PLD,PTS,NRR,Team_Cd,Projected PTS
0,1,Australia,8,14,1.000,AUS,14
1,2,India,7,11,0.854,IND,11
2,3,New Zealand,8,11,0.572,NZ,11
3,4,England,8,10,1.000,ENG,10
4,5,Pakistan,8,9,-0.792,PAK,9
5,6,Bangladesh,7,7,-0.133,BAN,7
6,7,Sri Lanka,7,6,-1.186,SL,6
7,8,South Africa,8,5,-0.080,SA,5
8,9,West Indies,7,3,-0.320,WI,3
9,10,Afghanistan,8,0,-1.418,AFG,0


In [130]:
points_table=points_table_df.set_index('Team')
points_table
for i in points_table_df['Team']:
     if i in win_list:
       points_table.loc[i, 'Projected PTS']=points_table.loc[i,'PTS']+2*win_list[i] 

points_table=points_table.sort_values(by='Projected PTS', ascending=False).reset_index()
points_table

,Team,Pos,PLD,PTS,NRR,Team_Cd,Projected PTS
0,India,2,7,11,0.854,IND,15
1,Australia,1,8,14,1.000,AUS,14
2,New Zealand,3,8,11,0.572,NZ,13
3,Pakistan,5,8,9,-0.792,PAK,11
4,England,4,8,10,1.000,ENG,10
5,Sri Lanka,7,7,6,-1.186,SL,8
6,Bangladesh,6,7,7,-0.133,BAN,7
7,South Africa,8,8,5,-0.080,SA,7
8,West Indies,9,7,3,-0.320,WI,5
9,Afghanistan,10,8,0,-1.418,AFG,0


In [188]:
knockoffs_df['Team1']=''
knockoffs_df['Team2']=''
#knockoffs_df.loc[0,'Team1']= points_table[0]
knockoffs_df.loc[0,'Team1'] = points_table.loc[0,'Team']
knockoffs_df.loc[0,'Team2'] = points_table.loc[3,'Team']
knockoffs_df.loc[1,'Team1'] = points_table.loc[1,'Team']
knockoffs_df.loc[1,'Team2'] = points_table.loc[2,'Team']
knockoffs_df

knockoff_match_df=sqlContext.createDataFrame(knockoffs_df)
knockoff_team1_df=knockoff_match_df.join(all_teams_metrics_df,knockoff_match_df.Team1 == all_teams_metrics_df.team,how='inner')
knockoff_team2_df=knockoff_match_df.join(all_teams_metrics_df,knockoff_match_df.Team2 == all_teams_metrics_df.team,how='inner')
knockoff_team1_df.createOrReplaceTempView("knockoff_team1_data")
knockoff_team2_df.createOrReplaceTempView("knockoff_team2_data")
knockoff_team_diff_df=sqlContext.sql("select t1.Date, t1.Venue, t1.No as match_no,t1.team,t1.Team1,t1.Team2,t1.Mat_avg-t2.Mat_avg diff_Mat_avg,t1.Inns_avg - t2.Inns_avg diff_Inns_avg,t1.NO_avg - t2.NO_avg as diff_NO_avg,t1.Runs_ave-t2.Runs_ave diff_Runs_avg, t1.Runs_max-t2.Runs_max	diff_Runs_max,t1.HS_avg-t2.HS_avg diff_HS_avg,t1.HS_max-t2.HS_max diff_HS_max,t1.Ave_max	-	t2.	Ave_max	diff_Ave_max,t1.Ave_avg	-	t2.	Ave_avg	diff_Ave_avg,t1.BF_avg	-	t2.	BF_avg	diff_BF_avg,t1.SR_avg	-	t2.	SR_avg	diff_SR_avg,t1.SR_max	-	t2.	SR_max	diff_SR_max,t1.bat_100_avg	-	t2.bat_100_avg	diff_100_avg,t1.bat_100_max	-	t2.bat_100_max	diff_100_max,t1.bat_50_ave	-	t2.bat_50_ave	diff_50_ave,t1.bat_50_max	-	t2.bat_50_max	diff_50_max,t1.bat_4s_avg	-	t2.bat_4s_avg	diff_4s_avg,t1.bat_4s_max	-	t2.bat_4s_max	diff_4s_max,t1.bat_6s_avg	-	t2.bat_6s_avg	diff_6s_avg,t1.bat_6s_max	-	t2.bat_6s_max	diff_6s_max,t1.Ct_ave	-	t2.	Ct_ave	diff_Ct_ave,t1.Ct_max	-	t2.	Ct_max	diff_Ct_max,t1.St_Avg	-	t2.	St_Avg	diff_St_Avg,t1.St_max	-	t2.	St_max	diff_St_max,t1.bowl_Mat_Avg	-	t2.	bowl_Mat_Avg	diff_bowl_Mat_Avg,t1.bowl_Inns_avg	-	t2.	bowl_Inns_avg	diff_bowl_Inns_avg,t1.bowl_maidens_avg	-	t2.	bowl_maidens_avg	diff_bowl_maidens_avg,t1.bowl_maidens_max	-	t2.	bowl_maidens_max	diff_bowl_maidens_max,t1.bowl_Runs_avg	-	t2.	bowl_Runs_avg	diff_bowl_Runs_avg,t1.bowl_Runs_max	-	t2.	bowl_Runs_max	diff_bowl_Runs_max,t1.bowl_Wkts_avg	-	t2.	bowl_Wkts_avg	diff_bowl_Wkts_avg,t1.bowl_Wkts_max	-	t2.	bowl_Wkts_max	diff_bowl_Wkts_max,t1.b_Ave_Avg	-	t2.	b_Ave_Avg	diff_b_Ave_Avg,t1.bowl_Econ_Avg	-	t2.	bowl_Econ_Avg	diff_bowl_Econ_Avg,t1.bowl_Econ_min	-	t2.	bowl_Econ_min	diff_bowl_Econ_min,t1.bowl_SR_Avg	-	t2.	bowl_SR_Avg	diff_bowl_SR_Avg,t1.bowl_SR_min	-	t2.	bowl_SR_min	diff_bowl_SR_min,t1.bowl_4w_avg	-	t2.	bowl_4w_avg	diff_bowl_4w_avg,t1.bowl_4w_max	-	t2.	bowl_4w_max	diff_bowl_4w_max,t1.bowl_5w_avg	-	t2.	bowl_5w_avg	diff_bowl_5w_avg,t1.bowl_5w_max	-	t2.	bowl_5w_max	diff_bowl_5w_max from knockoff_team1_data t1,knockoff_team2_data t2 where t1.No=t2.No order by t1.No")

semis_predictions = lr_PipelineModel.transform(knockoff_team_diff_df)
semis_predictions_df=predictions.select("Date","Venue","match_no","Team1","Team2","rawPrediction","probability", "prediction")
semis_win_predictions=semis_predictions_df.toPandas()
semis_win_list1=semis_win_predictions[(semis_win_predictions.prediction==0.0)]['Team2'].value_counts()
semis_win_list2=semis_win_predictions[(semis_win_predictions.prediction==1.0)]['Team1'].value_counts()
semis_win_list=semis_win_list1.append(semis_win_list2)
semis_win_predictions

,Date,Venue,match_no,Team1,Team2,rawPrediction,probability,prediction
0,2019-07-09,Old Trafford- Manchester,46,India,Pakistan,"[-0.29014198795654, 0.29014198795654]","[0.4279691064212501, 0.5720308935787499]",1.0
1,2019-07-11,Edgbaston- Birmingham,48,Australia,New Zealand,"[-1.0130888622576777, 1.0130888622576777]","[0.26637579070695794, 0.7336242092930421]",1.0


In [189]:
final_df['Team1']=semis_win_list.keys()[0]
final_df['Team2']=semis_win_list.keys()[1]


final_match_df=sqlContext.createDataFrame(final_df)
final_match_team1_df=final_match_df.join(all_teams_metrics_df,final_match_df.Team1 == all_teams_metrics_df.team,how='inner')
final_match_team2_df=final_match_df.join(all_teams_metrics_df,final_match_df.Team2 == all_teams_metrics_df.team,how='inner')
final_match_team1_df.createOrReplaceTempView("final_team1_data")
final_match_team2_df.createOrReplaceTempView("final_team2_data")
final_team_diff_df=sqlContext.sql("select t1.Date, t1.Venue, t1.No as match_no,t1.team,t1.Team1,t1.Team2,t1.Mat_avg-t2.Mat_avg diff_Mat_avg,t1.Inns_avg - t2.Inns_avg diff_Inns_avg,t1.NO_avg - t2.NO_avg as diff_NO_avg,t1.Runs_ave-t2.Runs_ave diff_Runs_avg, t1.Runs_max-t2.Runs_max	diff_Runs_max,t1.HS_avg-t2.HS_avg diff_HS_avg,t1.HS_max-t2.HS_max diff_HS_max,t1.Ave_max	-	t2.	Ave_max	diff_Ave_max,t1.Ave_avg	-	t2.	Ave_avg	diff_Ave_avg,t1.BF_avg	-	t2.	BF_avg	diff_BF_avg,t1.SR_avg	-	t2.	SR_avg	diff_SR_avg,t1.SR_max	-	t2.	SR_max	diff_SR_max,t1.bat_100_avg	-	t2.bat_100_avg	diff_100_avg,t1.bat_100_max	-	t2.bat_100_max	diff_100_max,t1.bat_50_ave	-	t2.bat_50_ave	diff_50_ave,t1.bat_50_max	-	t2.bat_50_max	diff_50_max,t1.bat_4s_avg	-	t2.bat_4s_avg	diff_4s_avg,t1.bat_4s_max	-	t2.bat_4s_max	diff_4s_max,t1.bat_6s_avg	-	t2.bat_6s_avg	diff_6s_avg,t1.bat_6s_max	-	t2.bat_6s_max	diff_6s_max,t1.Ct_ave	-	t2.	Ct_ave	diff_Ct_ave,t1.Ct_max	-	t2.	Ct_max	diff_Ct_max,t1.St_Avg	-	t2.	St_Avg	diff_St_Avg,t1.St_max	-	t2.	St_max	diff_St_max,t1.bowl_Mat_Avg	-	t2.	bowl_Mat_Avg	diff_bowl_Mat_Avg,t1.bowl_Inns_avg	-	t2.	bowl_Inns_avg	diff_bowl_Inns_avg,t1.bowl_maidens_avg	-	t2.	bowl_maidens_avg	diff_bowl_maidens_avg,t1.bowl_maidens_max	-	t2.	bowl_maidens_max	diff_bowl_maidens_max,t1.bowl_Runs_avg	-	t2.	bowl_Runs_avg	diff_bowl_Runs_avg,t1.bowl_Runs_max	-	t2.	bowl_Runs_max	diff_bowl_Runs_max,t1.bowl_Wkts_avg	-	t2.	bowl_Wkts_avg	diff_bowl_Wkts_avg,t1.bowl_Wkts_max	-	t2.	bowl_Wkts_max	diff_bowl_Wkts_max,t1.b_Ave_Avg	-	t2.	b_Ave_Avg	diff_b_Ave_Avg,t1.bowl_Econ_Avg	-	t2.	bowl_Econ_Avg	diff_bowl_Econ_Avg,t1.bowl_Econ_min	-	t2.	bowl_Econ_min	diff_bowl_Econ_min,t1.bowl_SR_Avg	-	t2.	bowl_SR_Avg	diff_bowl_SR_Avg,t1.bowl_SR_min	-	t2.	bowl_SR_min	diff_bowl_SR_min,t1.bowl_4w_avg	-	t2.	bowl_4w_avg	diff_bowl_4w_avg,t1.bowl_4w_max	-	t2.	bowl_4w_max	diff_bowl_4w_max,t1.bowl_5w_avg	-	t2.	bowl_5w_avg	diff_bowl_5w_avg,t1.bowl_5w_max	-	t2.	bowl_5w_max	diff_bowl_5w_max from final_team1_data t1,final_team2_data t2 where t1.No=t2.No order by t1.No")
predictions = lr_PipelineModel.transform(final_team_diff_df)
predictions_df=predictions.select("Date","Venue","match_no","Team1","Team2","rawPrediction","probability", "prediction")
win_predictions=predictions_df.toPandas()
win_list1=win_predictions[(win_predictions.prediction==0.0)]['Team2'].value_counts()
win_list2=win_predictions[(win_predictions.prediction==1.0)]['Team1'].value_counts()
win_list=win_list1.append(win_list2)
predictions_df=predictions.select("Date","Venue","match_no","Team1","Team2","rawPrediction","probability", "prediction")
predictions_df.toPandas()


,Date,Venue,match_no,Team1,Team2,rawPrediction,probability,prediction
0,2019-07-14,Lord’s- London,50,India,Australia,"[0.11779257523851167, -0.11779257523851167]","[0.529414141348514, 0.47058585865148606]",0.0


In [172]:
print("Winner of Cricket World Cup 2019 is %s"%(win_list.keys()[0]))

Winner of Cricket World Cup 2019 is Australia
